#Import Test Dataframe

In [23]:
import pandas as pd

df_stock = pd.read_csv(r'C:\Users\angel\Documents\Documents\GitHub\Group-Project-Hill-Wiley-Gwinn\Data\AAPL.csv')
df_fin = pd.read_csv(r"C:\Users\angel\Documents\Documents\GitHub\Group-Project-Hill-Wiley-Gwinn\Data\AAPLfinancials.csv")

#Import BackTrader and Instantiate Cerebro Engine

In [25]:
import backtrader as bt

cerebro = bt.Cerebro()

cerebro.broker.set_cash(2500)
print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.run()
print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

cerebro.adddata(data)


Starting Portfolio Value: 2500.00
Final Portfolio Value: 2500.00


#Create Strategy